#Tahap 3 – Case Retrieval

##Representasi Vektor

In [ ]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = vectorizer.fit_transform(df['ringkasan_fakta'].astype(str))

# BERT Embedding
from sentence_transformers import SentenceTransformer

bert_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
corpus = df['ringkasan_fakta'].astype(str).tolist()
corpus_embeddings = bert_model.encode(corpus, convert_to_tensor=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

##Splitting Data

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

##Model Retrieval

###TF-IDF + SVM

In [ ]:
from sklearn.svm import LinearSVC

svm_model = LinearSVC()
svm_model.fit(tfidf_matrix, df['case_id'].astype(str))

LinearSVC()

##Fungsi Retrieval

### TF-IDF + SVM

In [ ]:
def retrieve_svm(query: str):
    query_vec = vectorizer.transform([query])
    pred = svm_model.predict(query_vec)
    return pred[0]

### BERT

In [ ]:
from sentence_transformers import util

def retrieve_bert(query: str, k: int = 5):
    query_embedding = bert_model.encode(query, convert_to_tensor=True)
    similarities = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_k_idx = similarities.argsort(descending=True)[:k]
    return df.iloc[top_k_idx][['case_id', 'ringkasan_fakta']].to_dict(orient='records')

##Pengujian Awal

In [ ]:
test_queries = [
    {"query_id": 1, "query_text": "putusan dari mahkamah agung republik indonesia yang akan terus disempurnakan ke depannya", "ground_truth": "1"},
    {"query_id": 2, "query_text": "para penggugat telah mengajukan gugatannya tertanggal 26 agustus 2020 dan terdaftar di kepaniteraan pengadilan tata usaha negara bandung ", "ground_truth": "2"},
    {"query_id": 3, "query_text": "penggugat dengan surat gugatannya tertanggal 11 november 2020telah mengajukan gugatan yang didaftarkan di kepaniteraan pengadilan tata usahanegara bandung ", "ground_truth": "3"},
    {"query_id": 4, "query_text": "para penggugat telah mengajukan gugatantertanggal 8 desember 2020 yang terdaftar di kepaniteraan pengadilantata usaha negara bandung", "ground_truth": "4"},
    {"query_id": 5, "query_text": "penggugat telah mengajukan gugatannya tertanggal 27 juli 2020 dan telah didaftarkan di kepaniteraan pengadilan tata usaha negara bandung", "ground_truth": "5"},
    {"query_id": 6, "query_text": "objek gugatan dalam perkara a quo telah memenuhi syarat objekgugatan sebagaimana ketentuan pasal 1 angka (9) undang-undang nomor 51tahun 2009 ", "ground_truth": "6"},
    {"query_id": 7, "query_text": "penggugat telah mengajukan surat gugatan tertanggal 27agustus 2018 yang didaftarkan di kepaniteraan pengadilan tata usahanegara bandung tanggal14 september 2020", "ground_truth": "7"},
    {"query_id": 8, "query_text": "penggugat melalui kuasa hukumnya mengajukan gugatan, yang didaftarkan secara elektronik  di kepani-teraan pengadilan tata usaha negara bandung", "ground_truth": "8"},
    {"query_id": 9, "query_text": "penggugat melalui kuasa hukumnya mengajukan gugatan, tanggal 27 agustus 2020, didaftarkan secara elektronik  di kepani-teraan pengadilan tata usaha negara bandung ", "ground_truth": "9"},
    {"query_id": 10, "query_text": "terhadap gugatan para penggugat tersebut, tergugattelah mengajukan jawabannya tertanggal 17 november 2020, padapersidangan tanggal 19 november 2020", "ground_truth": "10"}
]

os.makedirs('/content/drive/MyDrive/Penalaran Komputer/data/eval', exist_ok=True)
import json
with open('/content/drive/MyDrive/Penalaran Komputer/data/eval/queries.json', 'w') as f:
    json.dump(test_queries, f, indent=2)